# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd9d81a1d90>
├── 'a' --> tensor([[ 0.3655, -0.8296,  1.4827],
│                   [ 0.5637, -0.1485, -1.1505]])
└── 'x' --> <FastTreeValue 0x7fd9d819a760>
    └── 'c' --> tensor([[ 0.7795, -0.6993, -0.1945,  0.1362],
                        [-0.3711,  1.2983, -0.2928,  0.8376],
                        [ 2.1103,  0.4199, -3.4788, -0.7593]])

In [4]:
t.a

tensor([[ 0.3655, -0.8296,  1.4827],
        [ 0.5637, -0.1485, -1.1505]])

In [5]:
%timeit t.a

61.5 ns ± 0.0262 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd9d81a1d90>
├── 'a' --> tensor([[-0.0331,  1.0394,  0.9173],
│                   [-1.0225, -1.4047,  0.1485]])
└── 'x' --> <FastTreeValue 0x7fd9d819a760>
    └── 'c' --> tensor([[ 0.7795, -0.6993, -0.1945,  0.1362],
                        [-0.3711,  1.2983, -0.2928,  0.8376],
                        [ 2.1103,  0.4199, -3.4788, -0.7593]])

In [7]:
%timeit t.a = new_value

59.8 ns ± 0.0257 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3655, -0.8296,  1.4827],
               [ 0.5637, -0.1485, -1.1505]]),
    x: Batch(
           c: tensor([[ 0.7795, -0.6993, -0.1945,  0.1362],
                      [-0.3711,  1.2983, -0.2928,  0.8376],
                      [ 2.1103,  0.4199, -3.4788, -0.7593]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3655, -0.8296,  1.4827],
        [ 0.5637, -0.1485, -1.1505]])

In [11]:
%timeit b.a

58.7 ns ± 0.317 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.1897,  0.1998, -0.5254],
               [ 0.3614,  1.4583,  0.9591]]),
    x: Batch(
           c: tensor([[ 0.7795, -0.6993, -0.1945,  0.1362],
                      [-0.3711,  1.2983, -0.2928,  0.8376],
                      [ 2.1103,  0.4199, -3.4788, -0.7593]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 1.04 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 57.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 556 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 506 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd921461e50>
├── 'a' --> tensor([[[ 0.3655, -0.8296,  1.4827],
│                    [ 0.5637, -0.1485, -1.1505]],
│           
│                   [[ 0.3655, -0.8296,  1.4827],
│                    [ 0.5637, -0.1485, -1.1505]],
│           
│                   [[ 0.3655, -0.8296,  1.4827],
│                    [ 0.5637, -0.1485, -1.1505]],
│           
│                   [[ 0.3655, -0.8296,  1.4827],
│                    [ 0.5637, -0.1485, -1.1505]],
│           
│                   [[ 0.3655, -0.8296,  1.4827],
│                    [ 0.5637, -0.1485, -1.1505]],
│           
│                   [[ 0.3655, -0.8296,  1.4827],
│                    [ 0.5637, -0.1485, -1.1505]],
│           
│                   [[ 0.3655, -0.8296,  1.4827],
│                    [ 0.5637, -0.1485, -1.1505]],
│           
│                   [[ 0.3655, -0.8296,  1.4827],
│                    [ 0.5637, -0.1485, -1.1505]]])
└── 'x' --> <FastTreeValue 0x7fd921461dc0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 62.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd9280b2ee0>
├── 'a' --> tensor([[ 0.3655, -0.8296,  1.4827],
│                   [ 0.5637, -0.1485, -1.1505],
│                   [ 0.3655, -0.8296,  1.4827],
│                   [ 0.5637, -0.1485, -1.1505],
│                   [ 0.3655, -0.8296,  1.4827],
│                   [ 0.5637, -0.1485, -1.1505],
│                   [ 0.3655, -0.8296,  1.4827],
│                   [ 0.5637, -0.1485, -1.1505],
│                   [ 0.3655, -0.8296,  1.4827],
│                   [ 0.5637, -0.1485, -1.1505],
│                   [ 0.3655, -0.8296,  1.4827],
│                   [ 0.5637, -0.1485, -1.1505],
│                   [ 0.3655, -0.8296,  1.4827],
│                   [ 0.5637, -0.1485, -1.1505],
│                   [ 0.3655, -0.8296,  1.4827],
│                   [ 0.5637, -0.1485, -1.1505]])
└── 'x' --> <FastTreeValue 0x7fd932b67f70>
    └── 'c' --> tensor([[ 0.7795, -0.6993, -0.1945,  0.1362],
                        [-0.3711,  1.2983, -0.2928,  0.8376],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 29.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.2 µs ± 340 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7795, -0.6993, -0.1945,  0.1362],
                       [-0.3711,  1.2983, -0.2928,  0.8376],
                       [ 2.1103,  0.4199, -3.4788, -0.7593]],
              
                      [[ 0.7795, -0.6993, -0.1945,  0.1362],
                       [-0.3711,  1.2983, -0.2928,  0.8376],
                       [ 2.1103,  0.4199, -3.4788, -0.7593]],
              
                      [[ 0.7795, -0.6993, -0.1945,  0.1362],
                       [-0.3711,  1.2983, -0.2928,  0.8376],
                       [ 2.1103,  0.4199, -3.4788, -0.7593]],
              
                      [[ 0.7795, -0.6993, -0.1945,  0.1362],
                       [-0.3711,  1.2983, -0.2928,  0.8376],
                       [ 2.1103,  0.4199, -3.4788, -0.7593]],
              
                      [[ 0.7795, -0.6993, -0.1945,  0.1362],
                       [-0.3711,  1.2983, -0.2928,  0.8376],
                       [ 2.1103,  0.4199, -3.4788, -0.7593]],

In [26]:
%timeit Batch.stack(batches)

77.4 µs ± 96.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7795, -0.6993, -0.1945,  0.1362],
                      [-0.3711,  1.2983, -0.2928,  0.8376],
                      [ 2.1103,  0.4199, -3.4788, -0.7593],
                      [ 0.7795, -0.6993, -0.1945,  0.1362],
                      [-0.3711,  1.2983, -0.2928,  0.8376],
                      [ 2.1103,  0.4199, -3.4788, -0.7593],
                      [ 0.7795, -0.6993, -0.1945,  0.1362],
                      [-0.3711,  1.2983, -0.2928,  0.8376],
                      [ 2.1103,  0.4199, -3.4788, -0.7593],
                      [ 0.7795, -0.6993, -0.1945,  0.1362],
                      [-0.3711,  1.2983, -0.2928,  0.8376],
                      [ 2.1103,  0.4199, -3.4788, -0.7593],
                      [ 0.7795, -0.6993, -0.1945,  0.1362],
                      [-0.3711,  1.2983, -0.2928,  0.8376],
                      [ 2.1103,  0.4199, -3.4788, -0.7593],
                      [ 0.7795, -0.6993, -0.1945,  0.1362],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

320 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
